# Assignment 1 ( Spell Correction )
## Lecture Assignment
Given a dictionary D and a spelling error corpus C for the English language, calculate the average success at
k (s@k) for the minimum edit distance (MED) algorithm for all misspelled tokens in C. 

a) Use WordNet1 as the dictionary D. A Python interface to WordNet is PyDictionary2 which is available
opensource3.  
b) Use Birkbeck4 spelling error corpus. This corpus includes the most common misspelled tokens and the
correct spell in pairs. For instance (‘desing’, ‘design’).  
c) Success at k (s@k) measures whether the correct spell of the token happens to be in the top-k (most similar,
least distant) list of tokens that are retrieved by the MED from the dictionary D. For instance, given ‘desing’
from Birkbeck corpus, the top-5 most similar (least distant) tokens to ‘desing’ based on MED from WordNet
are [‘desi’, ‘design’, ‘designer’, ‘designate’, ‘despair’]. Then, s@1 is 0 since the correct spell from Birkbeck
is ‘design’ which is not happening at the first item. However, s@k for k≥2 is 1. Report the average s@k for
k={1, 5, 10} using PyTrec_Eval_Terrier5.  
d) Comparing each misspelled word with all words in a dictionary takes time. You have to come up with
workarounds such as parallel runs.  

In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from heapq import nlargest
from concurrent.futures import ThreadPoolExecutor # for parallel distance calculation
import concurrent.futures
import time
import pytrec_eval
import json

### LevenshteinDistance
I used this link to get more insight about it : https://www.scaler.com/topics/levenshtein-distance-python/

In [ ]:
def levenshteinDistance(A, B):
    if len(A) > len(B):
        A, B = B, A  # Ensuring A is the smaller string

    N, M = len(A), len(B)
    prev_row = list(range(M + 1))

    for i in range(1, N + 1):
        current_row = [i]
        # two one-dimensional arrays (prev_row and current_row), significantly reducing memory usage
        for j in range(1, M + 1):
            # I would intentionally write the following lines of code like Piecewise Function of Levenshtein for more readability
            insertions = prev_row[j] + 1
            deletions = current_row[j - 1] + 1
            substitutions = prev_row[j - 1] + (A[i - 1] != B[j - 1])
            current_row.append(min(insertions, deletions, substitutions))
        prev_row = current_row

    return prev_row[-1]


## Necessary functions

We need some functions to make our work easier for retrieving words from our dictionary.
As you can see for the filter words function, we assumed that similar words should not be different in terms of length so we put some restrictions on it. This will help us to reduce the amount of time we need for running our code

In [ ]:

# Since going through all the words in a dictionary is not sufficient, we are going to put restrictions on it
def filter_words(token, max_length_diff=3):
    special_characters = set('!@#$%^&()-+=.:?/;][{}|\~`1234567890')
    length = len(token)
    # Restriction
    min_length, max_length = length - max_length_diff, length + max_length_diff
    return [w for w in wn.words(lang='eng') # getting the word from wordnet
            if min_length <= len(w) <= max_length 
            and not special_characters.intersection(w)]


def find_closest_match(token, top_n=10):
    words_chklst = filter_words(token)
    distances = []

    with ThreadPoolExecutor() as executor: # speed up the process ( OPTIMIZATION )
        futures = {executor.submit(levenshteinDistance, token, w): w for w in words_chklst}
        for future in futures:
            w = futures[future]
            try:
                distance = future.result()
                distances.append((distance, w))
            except Exception as exc:
                print(f"{w} generated an exception: {exc}")

    return nlargest(top_n, distances, key=lambda x: -x[0])

def return_top_1_5_10_words(da):
    token, correct_spelling = da
    token_distance = find_closest_match(token, 10)
    result = {
        'incorrect': token,
        'correct': correct_spelling,
        1: token_distance[:1],
        5: token_distance[:5],
        10: token_distance
    }
    return result

def return_success_at_k(result_list):
    success_dict = {1: [], 5: [], 10: []}
    for d in result_list:
        crr = d['correct']
        for k in [1, 5, 10]:
            success_dict[k].append(int(crr in [w[1] for w in d[k]]))

    return success_dict


def update_results_eval(results_eval, result, k, divisor):
    for word in result[k]:
        results_eval[word] = results_eval.get(word, 0) + 1 / divisor

# Example Usage
data = [("design", "design"), ("algoritm", "algorithm")]
results = [return_top_1_5_10_words(da) for da in data]
success = return_success_at_k(results)
print(success)


## Birkbeck
Since Birkbeck has so many files and working with them could be confusing, I instead used the preprocessed version of it and you can find it in this **<a href="https://www.dcs.bbk.ac.uk/~roger/corpora.html">link</a>**.  
You can find some information about the data in here:  
* The birkbeck file contains 36,133 misspellings of 6,136 words. It is an amalgamation of errors taken from the native-speaker section (British or American writers) of the Birkbeck spelling error corpus, a collection of files of spelling errors gathered from various sources, available as separate files with detailed documentation from the Oxford Text Archive. It includes the results of spelling tests and errors from free writing, taken mostly from schoolchildren, university students or adult literacy students. Most of them were originally handwritten.

* Each **correct** word is preceded by a `dollar` sign and followed by its misspellings, each on one line, without duplicates. (A misspelling might appear more than once in the corpus, but only as a misspelling of different words.) Where the spelling or misspelling contained a space, this has been replaced by an underscore (a_lot, Christ_mas). While most of the misspellings are non-words, there are also some real-word errors, such as "omitted" for "admitted".

* Correct spellings of dictionary words are given in Oxford English form. Where the misspellings were taken from American writers, attempts at specifically American forms (color, theater etc.) have been excluded. Where a correct American form appears as a misspelling, it represents a British writer's attempt at the British form, such as "color" for "colour". Apart from dictionary words, the correct spellings also contain some proper nouns, abbreviations, words with apostrophes or hyphens, made-up words and two-word items (e.g. "too much") where the misspelling was a single string ("tomuch").

* Users of this corpus should bear in mind that it includes the efforts of young children and extremely poor spellers being subjected to spelling tests way beyond their ability, so some of the misspellings are very different from their targets; the single letter "o", for example, appears as a misspelling of the word "accordingly".



In [ ]:
# Getting the correct and incorrect words from Wordnet dictionary 
corr = []
incorr = []
total_word_count = 0  # Initialize word count

with open('data/missp.txt', 'r') as data:
    for line in data:
        line = line.strip().lower()  # Processing the line
        words = line.split()  # Splitting the line into words
        total_word_count += len(words)  # Adding the number of words in the line to the total count

        if line.startswith('$'):
            current_correct = line[1:]  # Processing for correct words
        else:
            corr.append(current_correct)
            incorr.append(line)

# total_word_count now holds the total number of words in the file
print("Total number of words:", total_word_count)


In [ ]:
corr[5: 10]

In [ ]:
incorr[5: 10]

In [ ]:
# Since testing on all words is not sufficient we just explore some random words 
correct = corr[5: 10]
incorrect = incorr[5: 10]

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = list(executor.map(return_top_1_5_10_words, zip(incorrect, correct)))

# Diagnostic print statements
print("Number of results:", len(results))
print("First few results:", results[:5])

In [ ]:


# Assuming 'results' is prepared correctly
success_at_k = return_success_at_k(results)

# Prepare the query and results_eval dictionaries
query = {result["incorrect"]: {result["correct"]: 1} for result in results}
results_eval = {}

for result in results:
    incorrect_word = result["incorrect"]
    results_eval[incorrect_word] = {}

    # Update results for top 1, 5, and 10 words
    for k in [1, 5, 10]:
        divisor = k if k > 1 else 1  # Avoid division by zero
        for word in result[k]:
            if word not in results_eval[incorrect_word]:
                results_eval[incorrect_word][word] = 1 / divisor

# Evaluation using pytrec_eval
evaluator = pytrec_eval.RelevanceEvaluator(query, {'success'})
evaluated_results = evaluator.evaluate(results_eval)

# Printing a subset of the evaluated results
subset_keys = list(evaluated_results.keys())[:5]  # Adjust as needed
subset_evaluated_results = {k: evaluated_results[k] for k in subset_keys}
print(json.dumps(subset_evaluated_results, indent=1))

# Aggregate measures for all queries
aggregated_measures = {}
for measure in next(iter(evaluated_results.values())).keys():
    aggregated_measures[measure] = pytrec_eval.compute_aggregated_measure(
        measure,
        [query_measures[measure] for query_measures in evaluated_results.values()]
    )

# Print aggregated measures
for measure, avg in aggregated_measures.items():
    print(f"Average Success {measure}: {avg}")
